# Landcover conversion matrix

As part of my Markov chains project I searched for an interesting dataset to use it. This notebook is written in Julia. In particular, because Python was way too slow to calculate the count over 129600 x 64800 images. I also played with Spark on Azure around but Julia was just quicker to developed. 
The dataset is the annual landcover data from 1992 to 2015 by ESA CCI-LC project. The idea is to calculate the probabilites from one landcover type to another.

In [37]:
using ArchGDAL
using CSV, DataFrames

The function **getlandcoverchange()** reads the file with ArchGDAL. Then it reads two succesive raster bands which represents the landcover for a certain year. We pair the landcover types of the same pixel (same longitude and latitude) and uses it as key. Then we increased the key by one. As two uncompressed maps are too big to fit in memory I had to use the windowing method of ArchGDAL. The result is a dictionary with the count of all the possible pairs of landcover type over the data time period.

In [35]:
function getlandcoverchange()
    landcover = [0, 10, 11, 12, 20, 30, 40, 50, 60, 61, 62, 70, 
    71, 72, 80, 81, 82, 90, 100, 110, 120, 121, 122, 130, 
    140, 150, 151, 152, 153, 160, 170, 180, 190, 200, 201, 202, 
    210, 220];
    landcoverchangedict = Dict((i,j) => 0 for i in landcover for j in landcover);

    ArchGDAL.registerdrivers() do
        ArchGDAL.read("/run/media/torben/Data/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992_2015-v2.0.7.tif") do dataset
            println(dataset)
            println("Computation started")
            for i in 1:ArchGDAL.nraster(dataset)-1          
                band_1 = ArchGDAL.getband(dataset, i);
                band_2 = ArchGDAL.getband(dataset, i+1);
                @time for (cols,rows) in ArchGDAL.windows(band_1)
                    zipped_bands = zip(ArchGDAL.read(band_1, rows, cols), ArchGDAL.read(band_2, rows, cols));
                    map(zipped_cell -> landcoverchangedict[zipped_cell] += 1, zipped_bands)
                                
                end
                println(i, "/", ArchGDAL.nraster(dataset)-1, "completed")
            end
        end
    end
    return landcoverchangedict;
end

getlandcoverchange (generic function with 1 method)

I created two useless functions write the resulting dictionary into a text file and read it. I should replace it with functions of the CSV.jl module. The function **getlandcoverchange()** took over 4 hours because each iteration took ~10-11 minutes and allocated 78 GB memory according to @time.

In [69]:
function write_result(res)
    open("res.txt", "w") do file
        write(file, "lc_type, count\n")
        for (k, v) in res
            write(file, "$k,$v\n")
        end
    end
end

write_result (generic function with 1 method)

In [68]:
function parse_countfile(data)
    splitted = split(data, "\n")
    res = Dict{Tuple{Int, Int}, Int}()
    for line in splitted[2:end-1]
        tuple = split(line, ",")
        res[(parse(Int, tuple[1][2:end]), parse(Int, tuple[2][1:end-1]))] = parse(Int, tuple[3])
    end
    return res
end

parse_countfile (generic function with 1 method)

If we had run the function and saved the dicitonary with the function write_result we will open the file. Otherwise we run **getlandcoverchange()**.

In [39]:
try
    data = open("res.txt") do file
        read(file, String)
    end
    res = parse_countfile(data)
catch exception
    if isa(exception, SystemError)
        res = getlandcoverchange()
    end
end

In [52]:
landcover = [0, 10, 11, 12, 20, 30, 40, 50, 60, 61, 62, 70, 
    71, 72, 80, 81, 82, 90, 100, 110, 120, 121, 122, 130, 
    140, 150, 151, 152, 153, 160, 170, 180, 190, 200, 201, 202, 
    210, 220];

For each landcover type we calculated the count. I discovered that there is no pixel with no data with landcover type 0.

In [42]:
landcovercount = Dict(i => 0 for i in landcover);
for lc_type in landcover
    for key in keys(res)
        if lc_type == key[1]
            landcovercount[lc_type] += res[key]
        end
    end
end            

A small example how to calculate the probabilities for the conversion of one landcover type. This case: "Tree cover, broadleaved, deciduous, closed to open (>15%)"

In [66]:
landcover_num = 50
for (k, v) in res
    if k[1] == landcover_num
        println(k, " - ", v/landcovercount[landcover_num])
    end
end

(50, 122) - 5.6313506881012784e-5
(50, 140) - 0.0
(50, 100) - 0.0002770327021736787
(50, 153) - 1.0874152377259673e-8
(50, 40) - 0.0005828533246608467
(50, 30) - 0.0011438191554167454
(50, 152) - 0.0
(50, 121) - 4.544960727599453e-5
(50, 70) - 0.0
(50, 20) - 7.469299922905708e-6
(50, 62) - 1.2427602716868197e-9
(50, 61) - 3.293314719970072e-8
(50, 220) - 0.0
(50, 0) - 0.0
(50, 72) - 0.0
(50, 82) - 0.0
(50, 151) - 0.0
(50, 90) - 0.0
(50, 130) - 0.00012946237647249898
(50, 50) - 0.9965357976647312
(50, 71) - 0.0
(50, 110) - 2.769895162542376e-5
(50, 120) - 0.00026373268174608637
(50, 81) - 0.0
(50, 170) - 3.203835980408621e-6
(50, 12) - 2.386286135679447e-5
(50, 210) - 0.00011536761085116293
(50, 190) - 9.546573717030228e-6
(50, 202) - 0.0
(50, 200) - 7.698899883099849e-7
(50, 60) - 5.245349347715352e-5
(50, 10) - 0.00018096640110208377
(50, 150) - 1.2260140770258398e-5
(50, 80) - 3.4206976478179715e-7
(50, 160) - 0.00014047509662005173
(50, 11) - 0.00029692028411141495
(50, 180) - 9.415

Now we create the landcover conversion matrix and save it with CSV.jl and DataFrames.jl.

In [84]:
array = Array{Float64}(undef, length(landcover)-1, length(landcover)-1)
for (k, v) in res
    if k[1] == 0 || k[2] == 0
        continue
    end
    i = findfirst(isequal(k[1]), landcover) - 1
    j = findfirst(isequal(k[2]), landcover) - 1
    array[i,j] = v / landcovercount[k[1]]
end

In [92]:
CSV.write("landcover_matrix.csv", DataFrame(array))

"landcover_matrix.csv"